In [17]:
import pandas as pd
import os
import time

# CSV con Ns diferentes

In [3]:
input_file = "data.csv"
df = pd.read_csv(input_file)

Ns = [pow(2, i) * 1000 for i in [1, 2, 3, 4, 5, 6, 7]]

for i, size in enumerate(Ns, start=1):
    subset_df = df.head(size) 
    output_file = f"data{i}.csv"  
    subset_df.to_csv(output_file, index=False) 
    print(f"Archivo {output_file} creado con {size} registros.")

Archivo data1.csv creado con 2000 registros.
Archivo data2.csv creado con 4000 registros.
Archivo data3.csv creado con 8000 registros.
Archivo data4.csv creado con 16000 registros.
Archivo data5.csv creado con 32000 registros.
Archivo data6.csv creado con 64000 registros.
Archivo data7.csv creado con 128000 registros.


# Crear Indices

### Funciones Helper

In [22]:
from Spimi import SPIMIIndex

# Uso del índice
chunk_size = 10000  # Cantidad de filas por chunk
memory_limit = 500000  # Límite de memoria en bytes

# Función generadora para leer los archivos en chunks
def document_chunks(file_path, chunk_size):
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        yield chunk["merge"].dropna().tolist()

# Eliminar archivos si existen
def delete_previous_indices():
    files_to_delete = ["final_index.txt", "index_pointer.txt"]
    for file in files_to_delete:
        if os.path.exists(file):
            os.remove(file)

# Crear Indices y Hacer Consultas

Los tiempos se toman solo en las consultas, no en la creacion del indice. 

In [11]:
query = "Tanjirou Kimetsu Nezuko"
topK = 10

In [27]:
# Crear índices para cada archivo generado
for i in range(1, 8):  # Iterar por los nombres de los archivos generados

    # Se eliminan el indice para la creacion del siguiente.
    delete_previous_indices()

    file_path = f"data{i}.csv"
    print(f"Creando índice para {file_path}...")

    # Inicializar el índice
    index = SPIMIIndex(memory_limit=memory_limit)

    # Construir el índice utilizando los chunks del archivo actual
    index.construct_index(document_chunks(file_path, chunk_size))
    print(f"Índice creado para {file_path}.")

    # Iniciar Tiempo
    start_time = time.time()

    # Se busca la query en el indice
    query_index = index.retrieve_index(query)
    data = dict(list(query_index.items())[:topK])

    # Se comparan los resultados en el archivo csv
    df = pd.read_csv(file_path)
    results = [df.loc[doc_id, "merge"] for doc_id in list(query_index.keys())[:topK]]

    # Finalizar Tiempo
    end_time = time.time()
    query_time = round((end_time - start_time) * 1000, 3)
    print(f"{i}: {query_time} ms")

    



Creando índice para data1.csv...
Indexing...
collection_size 2000
Merging blocks...
Índice creado para data1.csv.
Query: ['tanjir', 'kimetsu', 'nezuko']


1: 77.699 ms
Creando índice para data2.csv...
Indexing...
collection_size 4000
Merging blocks...
Índice creado para data2.csv.
Query: ['tanjir', 'kimetsu', 'nezuko']


2: 116.502 ms
Creando índice para data3.csv...
Indexing...
collection_size 8000
Merging blocks...
Índice creado para data3.csv.
Query: ['tanjir', 'kimetsu', 'nezuko']


3: 153.773 ms
Creando índice para data4.csv...
Indexing...
collection_size 16000
Merging blocks...
Índice creado para data4.csv.
Query: ['tanjir', 'kimetsu', 'nezuko']


4: 275.077 ms
Creando índice para data5.csv...
Indexing...
collection_size 32000
Merging blocks...
Índice creado para data5.csv.
Query: ['tanjir', 'kimetsu', 'nezuko']


5: 386.998 ms
Creando índice para data6.csv...
Indexing...
collection_size 64000
Merging blocks...
Índice creado para data6.csv.
Query: ['tanjir', 'kimetsu', 'nezuko']
